<a href="https://colab.research.google.com/github/ImajiPietersz/FYP/blob/main/NASA_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"imajipietersz","key":"a867e2513f7abed17b6257011d4c5467"}'}

In [7]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!ls -lha kaggle.json

!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory

-rw-r--r-- 1 root root 69 May  4 13:28 kaggle.json
/content


In [9]:
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d vinayak123tyagi/bearing-dataset

100% 1.56G/1.56G [00:48<00:00, 30.6MB/s]
100% 1.56G/1.56G [00:48<00:00, 34.1MB/s]


In [11]:
from zipfile import ZipFile
file_name = "bearing-dataset.zip"

with ZipFile(file_name, 'r') as zip1:
  zip1.extractall()
  print('Done')

Done


In [12]:
import numpy as np
import os
import re
import scipy.io as scio

!pip install tensorflow-addons

import numpy as np
import pandas as pd
import scipy.signal
from keras.models import Sequential,Model,load_model
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer,OrdinalEncoder, OneHotEncoder
from sklearn import preprocessing
from keras.layers import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import itertools
from sklearn.metrics import confusion_matrix
from keras.callbacks import ReduceLROnPlateau
from keras import backend as k
from keras.callbacks import ModelCheckpoint
from datetime import datetime
from tqdm import tqdm
import tensorflow_addons as tfa
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
B1 ={
    "early" : "2003-10-23 09:14:13",
    "suspect" : "2003-11-08 12:11:44",
    "normal" : "2003-11-19 21:06:07",
    "suspect_1" : "2003-11-24 20:47:32" ,
    "imminent_failure" : "2003-11-25 23:39:56"
}
B2 = {
    "early" : "2003-11-01 21:41:44",
    "normal" : "2003-11-24 01:01:24",
    "suspect" : "2003-11-25 10:47:32",
    "imminient_failure" : "2003-11-25 23:39:56"
}

B3 = {
    "early" :"2003-11-01 21:41:44",
    "normal" : ["2003-11-01 21:51:44" , "2003-11-22 09:16:56"],
    "suspect" : ["2003-11-22 09:26:56" , "2003-11-25 10:47:32"],
    "Inner_race_failure" : ["2003-11-25 10:57:32" , "2003-11-25 23:39:56"]
}

B4 = {
    "early" : ["2003-10-22 12:06:24" , "2003-10-29 21:39:46"],
    "normal" : ["2003-10-29 21:49:46" , "2003-11-15 05:08:46"],
    "suspect" : ["2003-11-15 05:18:46" , "2003-11-18 19:12:30"],
    "Rolling_element_failure" : ["2003-11-19 09:06:09" , "2003-11-22 17:36:56"],
    "Stage_two_failure" : ["2003-11-22 17:46:56" , "2003-11-25 23:39:56"]
}

def file2datetime(file_name):
    [y,m,d,H,M,S] = map(int,file_name.split("."))
    return datetime(y,m,d,H,M,S)

def datetime2file(date):
    return date.strftime("%Y.%m.%d.%H.%M.%S")

def select_label_b1(date_time):
    if date_time < datetime.strptime("2003-10-23 09:14:13", "%Y-%m-%d %H:%M:%S"):
        return "early"
    
    elif date_time < datetime.strptime("2003-11-08 12:11:44", "%Y-%m-%d %H:%M:%S"):
        return "suspect"
    
    elif date_time < datetime.strptime("2003-11-19 21:06:07", "%Y-%m-%d %H:%M:%S"):
        return "normal"
    
    elif date_time < datetime.strptime( "2003-11-24 20:47:32", "%Y-%m-%d %H:%M:%S"):
        return "suspect"
    
    elif date_time < datetime.strptime( "2003-11-25 23:39:56", "%Y-%m-%d %H:%M:%S"):
        return "imminent_failure"
    
def select_label_b2(date_time):
    if date_time < datetime.strptime("2003-11-01 21:41:44", "%Y-%m-%d %H:%M:%S"):
        return "early"
    
    elif date_time < datetime.strptime("2003-11-24 01:01:24", "%Y-%m-%d %H:%M:%S"):
        return "normal"
    
    elif date_time < datetime.strptime("2003-11-25 10:47:32", "%Y-%m-%d %H:%M:%S"):
        return "suspect"
    
    elif date_time < datetime.strptime( "2003-11-25 23:39:56", "%Y-%m-%d %H:%M:%S"):
        return "imminent_failure"
    
def select_label_b3(date_time):
    if date_time < datetime.strptime("2003-11-01 21:41:44", "%Y-%m-%d %H:%M:%S"):
        return "early"
    
    elif date_time < datetime.strptime("2003-11-22 09:16:56", "%Y-%m-%d %H:%M:%S"):
        return "normal"
    
    elif date_time < datetime.strptime("2003-11-25 10:47:32", "%Y-%m-%d %H:%M:%S"):
        return "suspect"
    
    elif date_time < datetime.strptime("2003-11-25 23:39:56", "%Y-%m-%d %H:%M:%S"):
        return "Inner_race_failure" 
    
def select_label_b4(date_time):
    if date_time < datetime.strptime( "2003-10-29 21:39:46", "%Y-%m-%d %H:%M:%S"):
        return "early"
    
    elif date_time < datetime.strptime("2003-11-15 05:08:46", "%Y-%m-%d %H:%M:%S"):
        return "normal"
    
    elif date_time < datetime.strptime("2003-11-18 19:12:30", "%Y-%m-%d %H:%M:%S"):
        return "suspect"
    
    elif date_time < datetime.strptime("2003-11-22 17:36:56", "%Y-%m-%d %H:%M:%S"):
        return "Rolling_element_failure"
    
    elif date_time < datetime.strptime("2003-11-25 23:39:56", "%Y-%m-%d %H:%M:%S"):
        return "Stage_two_failure"
    
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [14]:
class Data(object):
    def __init__(self,file_fol,channels=8,sample_rate=10,samples_per_file=20480):
        self.file_fol = file_fol
        self.channels = channels
        self.sample_rate = sample_rate
        self.samples_per_file = samples_per_file
        
    def file_list(self):
        fil_list = os.listdir(self.file_fol)
        filtered = [x for x in fil_list if ".ipynb" not in x]
        return filtered
    
    def get_data(self):
        file_list = self.file_list()
        """
        
        b1 = []
        b2 = []
        b3 = []
        b4 = []
        b1l = []
        b2l = []
        b3l = []
        b4l = []
        """
        sample_ind = np.arange(0,self.samples_per_file,self.sample_rate).tolist()
        for i,fil_name in tqdm(enumerate(file_list),total=len(file_list),desc="Reading Files"):
            __date = file2datetime(fil_name)
            fil_loc = os.path.join(self.file_fol,fil_name)
            arr = np.loadtxt(fil_loc).T
            
            __l = [select_label_b1(__date),select_label_b2(__date),select_label_b3(__date),select_label_b4(__date)]
            
            if __l[0] and __l[1] and __l[2] and __l[3]:
                if self.channels==4:
                    if i == 0:
                        full_arr = np.copy(arr[:,sample_ind])
                        labels = __l
                    else:
                        full_arr = np.vstack([full_arr,arr[:,sample_ind]])
                        labels += __l

                elif self.channels == 8:
                    __ind = np.arange(0,8,2).tolist()
                    if i == 0:
                        full_arr = np.copy(arr[__ind][:,sample_ind])
                        labels = __l
                    else:
                        full_arr = np.vstack([full_arr,arr[__ind][:,sample_ind]])
                        labels += __l
                else:
                    print("Error")
            
        return full_arr,np.array(labels)

In [15]:
Data = Data("1st_test/1st_test")
data,label = Data.get_data()

Reading Files: 100%|██████████| 2156/2156 [05:53<00:00,  6.10it/s]


In [16]:
label

array(['normal', 'normal', 'normal', ..., 'imminent_failure',
       'Inner_race_failure', 'Stage_two_failure'], dtype='<U23')

In [17]:
len(label)

8620

In [18]:
list_label=[]
for i in label:
  if i=='normal':
    list_label.append(1)
  elif i=='suspect':
    list_label.append(1)
  elif i=='early':
    list_label.append(1)
  elif i=='imminent_failure':
    list_label.append(-1)
  elif i=='Inner_race_failure':
    list_label.append(-1)
  elif i=='Stage_two_failure':
    list_label.append(-1)
  elif i=='Rolling_element_failure':
    list_label.append(-1)

  
len(list_label)


8620

In [19]:
import numpy
arr = numpy.array(list_label)

In [20]:
label

array(['normal', 'normal', 'normal', ..., 'imminent_failure',
       'Inner_race_failure', 'Stage_two_failure'], dtype='<U23')

In [30]:
df = pd.DataFrame(data)

In [31]:
df

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.215,-0.132,-0.151,0.000,-0.071,-0.115,-0.059,-0.149,0.085,-0.188,...,-0.125,-0.244,-0.068,-0.090,-0.132,-0.229,0.010,-0.166,-0.183,-0.164
1,-0.168,-0.237,-0.015,-0.183,-0.012,-0.249,-0.146,-0.190,-0.095,-0.076,...,-0.105,0.066,-0.193,-0.322,-0.017,-0.027,-0.156,-0.085,-0.103,-0.078
2,-0.076,-0.127,-0.259,-0.137,-0.234,-0.049,-0.071,-0.144,-0.078,-0.076,...,-0.181,-0.024,0.017,-0.305,-0.271,-0.039,-0.208,-0.154,-0.220,0.037
3,-0.247,-0.059,-0.127,-0.188,-0.149,-0.032,0.022,-0.142,-0.095,-0.020,...,-0.164,-0.054,0.024,-0.095,-0.176,-0.142,-0.139,-0.212,-0.154,-0.183
4,-0.142,-0.291,0.020,-0.161,-0.125,-0.117,-0.037,-0.068,-0.156,-0.266,...,-0.100,-0.244,-0.046,-0.115,-0.085,-0.295,-0.061,-0.242,-0.093,-0.115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8615,-0.183,-0.032,-0.061,-0.076,-0.017,-0.137,-0.195,-0.237,-0.105,-0.122,...,-0.117,0.159,-0.095,-0.032,-0.186,-0.112,-0.181,-0.063,-0.146,-0.061
8616,-0.344,-0.015,-0.117,0.044,0.144,-0.129,-0.066,-0.059,-0.203,-0.142,...,-0.024,0.037,-0.220,-0.134,-0.225,-0.054,-0.208,-0.093,-0.146,-0.015
8617,0.083,-0.203,-0.103,-0.342,-0.266,-0.115,-0.327,-0.032,-0.259,-0.066,...,0.037,-0.249,-0.164,0.012,-0.452,0.044,-0.188,-0.039,-0.317,0.159
8618,0.107,0.371,0.120,-0.332,0.022,-0.085,-0.208,-0.618,-0.508,-0.005,...,-0.337,0.229,0.029,-0.369,-0.083,0.139,-0.540,-1.602,0.295,-0.042


In [27]:
df

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,binaryLabel
0,-0.215,-0.132,-0.151,0.000,-0.071,-0.115,-0.059,-0.149,0.085,-0.188,...,-0.244,-0.068,-0.090,-0.132,-0.229,0.010,-0.166,-0.183,-0.164,1
1,-0.168,-0.237,-0.015,-0.183,-0.012,-0.249,-0.146,-0.190,-0.095,-0.076,...,0.066,-0.193,-0.322,-0.017,-0.027,-0.156,-0.085,-0.103,-0.078,1
2,-0.076,-0.127,-0.259,-0.137,-0.234,-0.049,-0.071,-0.144,-0.078,-0.076,...,-0.024,0.017,-0.305,-0.271,-0.039,-0.208,-0.154,-0.220,0.037,1
3,-0.247,-0.059,-0.127,-0.188,-0.149,-0.032,0.022,-0.142,-0.095,-0.020,...,-0.054,0.024,-0.095,-0.176,-0.142,-0.139,-0.212,-0.154,-0.183,1
4,-0.142,-0.291,0.020,-0.161,-0.125,-0.117,-0.037,-0.068,-0.156,-0.266,...,-0.244,-0.046,-0.115,-0.085,-0.295,-0.061,-0.242,-0.093,-0.115,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8615,-0.183,-0.032,-0.061,-0.076,-0.017,-0.137,-0.195,-0.237,-0.105,-0.122,...,0.159,-0.095,-0.032,-0.186,-0.112,-0.181,-0.063,-0.146,-0.061,1
8616,-0.344,-0.015,-0.117,0.044,0.144,-0.129,-0.066,-0.059,-0.203,-0.142,...,0.037,-0.220,-0.134,-0.225,-0.054,-0.208,-0.093,-0.146,-0.015,-1
8617,0.083,-0.203,-0.103,-0.342,-0.266,-0.115,-0.327,-0.032,-0.259,-0.066,...,-0.249,-0.164,0.012,-0.452,0.044,-0.188,-0.039,-0.317,0.159,-1
8618,0.107,0.371,0.120,-0.332,0.022,-0.085,-0.208,-0.618,-0.508,-0.005,...,0.229,0.029,-0.369,-0.083,0.139,-0.540,-1.602,0.295,-0.042,-1


In [28]:
df['binaryLabel'].value_counts()

 1    7771
-1     849
Name: binaryLabel, dtype: int64

In [35]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score

def splitTrainTest(ratio, data, labels):
  
  x_train, x_test,_ = np.split(data, [round(len(data) * ratio), len(data)])
  labels_train, labels_test,_ = np.split(labels, [round(len(labels) * ratio), len(labels)])

  return (x_train,labels_train, x_test,labels_test)

# Split the dataset into training and testing sets
X_train, y_train, X_test, y_test = splitTrainTest(0.7,df, arr)
# Create an instance of the OneClassSVM class
ocs = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)

# Fit the model to the training data
ocs.fit(X_train)

# Predict the labels of the testing data
y_pred = ocs.predict(X_test)
y_test = pd.Series(np.array(y_test)).replace({"Good": 1, "Faulty": -1})

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9323279195668986
